In [1]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import date

import pandas as pd

In [ ]:

def fetch_data(category: str):
    # URL страницы
    url = "https://whatstat.ru/channels/" + category

    # Заголовки для запроса (чтобы сайт не заблокировал бота)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
    }

    # Отправляем GET-запрос
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Проверка на ошибки

    # Парсим страницу
    soup = BeautifulSoup(response.text, "html.parser")

    # Находим таблицу
    table = soup.find("table")

    # Проверяем, что таблица найдена
    if not table:
        print("Таблица не найдена!")
        return None

    # Собираем заголовки таблицы
    data = []
    for row in table.find_all("tr")[1:]:  # Пропускаем заголовок
        cells = row.find_all("td")
        if cells:
            # category = "people_blogs"
            channel_name = cells[1].text.strip()
            
            # Ищем ссылку и извлекаем код после /channel/
            link = cells[1].find("a")
            channel_code = link["href"].split("/channel/")[-1] if link else "Нет данных"

            subscribers = cells[2].text.strip()
            views = cells[3].text.strip()

            data.append([category, channel_name, channel_code, subscribers, views])
    return data

In [ ]:
# data = [] 
categories = ['people_blogs', 'pets_animals', 'gaming',  'music', 'science_technology', 'news_politics', 'education', 'nonprofits_activism', 'travel_events', 'entertainment', 'sports', 'cars_vehicles', 'film_animation', 'howto_style', 'comedy']
for category in categories:
    print(category)
    d = fetch_data(category)
    if d:
        data.extend(d)

df = pd.DataFrame(data, columns=["Категория", "Канал", "Код канала", "Подписчики", "Просмотры"])
df.to_csv("whatstat_channels.csv", index=False, encoding="utf-8", sep='\t')

print("Данные успешно сохранены в whatstat_channels.csv")

pets_animals
gaming
music
science_technology
news_politics
education
nonprofits_activism
travel_events
entertainment
sports
cars_vehicles
film_animation
howto_style
comedy
Данные успешно сохранены в whatstat_channels.csv


In [10]:
duplicates = df[df.duplicated(subset=["Код канала"], keep=False)]
print(duplicates)


Empty DataFrame
Columns: [Категория, Канал, Код канала, Подписчики, Просмотры]
Index: []


In [ ]:


DATES = {"1 янв 2025 Ср": date(2025, 1, 1), "1 Фев 2025 Сб": date(2025, 2, 1), "1 мар 2025 Сб": date(2025, 3, 1)}

# Путь к chromedriver (замени на свой путь)
CHROMEDRIVER_PATH = "C:/chromedriver-win64/chromedriver.exe"



def fetch_channel_stat(channel_ids: list):
    data = []
    # Запуск Selenium
    service = Service(CHROMEDRIVER_PATH)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Запуск без графического интерфейса (необязательно)
    driver = webdriver.Chrome(service=service, options=options)
    i = 0
    try:
        for channel_id in channel_ids:
            i += 1
            if (i % round(len(channel_ids)/20)) == 0:
                print(round(i/len(channel_ids)*100), '%')
            url = f"https://whatstat.ru/channel/{channel_id}"
            # Открываем страницу
            driver.get(url)

            # Ожидаем загрузки страницы
            wait = WebDriverWait(driver, 10)
            button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.load.btn.btn-outline-primary.btn-sm[value="90"]')))
            driver.execute_script("arguments[0].click();", button)  # Используем JS-клик на случай блокировки обычного клика
            time.sleep(2)  # Ждем подгрузку данных

            # Парсим таблицу статистики
            table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "table-stats")))
            rows = table.find_elements(By.TAG_NAME, "tr")

            # Собираем данные
            for row in rows[1:]:  # Пропускаем заголовок
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) >= 3:
                    period = DATES.get(cells[0].text.strip())
                    if not period: continue
                    
                    try:
                        subscribers = int(cells[1].text.strip().split("\n")[0].replace(" ", ""))
                    except ValueError:
                        subscribers = None  # Если число отсутствует или в некорректном формате
                    try:
                        views = int(cells[2].text.strip().split("\n")[0].replace(" ", ""))
                    except ValueError:
                        views = None  # Если число отсутствует или в некорректном формате
    
                    
                    data.append([channel_id, period, subscribers, views])

    except Exception as e:
        print(f"Ошибка {i}: {url=} {e}")
    finally:
        # Закрываем браузер
        driver.quit()
        
    return data

# ch_ids = [ch[2] for ch in data]
# from 500 Ошибка 1704: url='https://whatstat.ru/channel/Andysmobile'
# stat_data.extend(fetch_channel_stat(ch_ids[5500:]))
# print(len(stat_data))
# dfs = pd.DataFrame(stat_data, columns=["channel_id", "period", "subscribers", "views"])
# dfs.to_csv("whatstat_channels_stat.csv", index=False, encoding="utf-8", sep='\t')


In [14]:
import pandas as pd
from app.channel import ChannelDAO
import nest_asyncio
nest_asyncio.apply()

if 0:
    df = pd.read_csv("logs/whatstat_channels.csv", sep='\t')
    # ch_ids = df["Код канала"].tolist()
    print(df.columns)

    channels_db = await ChannelDAO.find_all()
    print(len(channels_db))
    df_db = pd.DataFrame(channels_db)

    # Объединяем DataFrame по полю "Код канала" и "channel_id"
    df = df.merge(df_db, left_on="channel_id", right_on="channel_id", how="left")
    # Оставляем только нужные столбцы (если не нужен "channel_id", можно его убрать)
    df = df[['category', 'channel', 'channel_id', 'category_id', 'status']]
    
# f_df = df[df["status"].isna() | (df["status"] == 0)]
# f_df = df[(df["category"]== 'news_politics') | (df["category"] == 'nonprofits_activism')]
f_df = df[(df["category"]== 'film_animation')]
print(len(df))
print(len(f_df))

7500
500


In [7]:
print(len(f_df))
f_df.head(3)

851


,category,channel_title,channel_id,category_id,status
2504,news_politics,Телеканал Прямий,UCH9H_b9oJtSHBovh94yB5HA,1,NaN
2505,news_politics,УНІАН,UCKCVeAihEfJr-pGH7B73Wyg,1,NaN
2507,news_politics,Фабрика новин,UCMp5Buw-6LpbbV9r9Sl_5yg,1,NaN


In [15]:
category_match ={
    "people_blogs": 10,
    "pets_animals": 2,
    "gaming": 12,
    "music": 13,
    "science_technology": 14,
    "news_politics": 1, # 1
    "education": 15,
    "nonprofits_activism": 1, # 1
    "travel_events": 16,
    "entertainment": 17,
    "sports": 18,
    "cars_vehicles": 5,
    "film_animation": 6, #6
    "howto_style": 11,
    "comedy": 3,
}



f_df.loc[:, "category_id"] = f_df["category"].map(category_match)
f_df = f_df[f_df["channel_id"].str.startswith("UC")]
f_df

,category,channel,channel_id,category_id,status
6009,film_animation,DareX,UCb1WBEEfA4e6FOLOKAH5j8w,6.0,NaN
6013,film_animation,Три Кота,UCBZNnwQOBirwpeWncqTj_KQ,6.0,NaN
6016,film_animation,WB Kids International,UCwKa3PzQeReoqMqNkDgG-8w,6.0,NaN
6018,film_animation,ZAKATOON,UCNVy3JfDYZ8jgsipemTMiGA,6.0,NaN
6019,film_animation,КиноМУР,UCyN8Cpug_YwyAKCXbJ9jXrg,6.0,NaN
...,...,...,...,...,...
6492,film_animation,Hot Wheels Россия,UCUr0P6x8O9qzMyYrsiK9erA,6.0,NaN
6493,film_animation,"Спокойной ночи, малыши!",UCoVS2PU1QJpXrO9oTyke2cw,6.0,NaN
6494,film_animation,Pastime,UCxSZ0B_GUU4nNECXZedgy9w,6.0,NaN
6497,film_animation,Мурвел,UClS7qgoaan8I0HgyrFpkXiw,6.0,NaN


In [16]:
# f_df.[:, "status"] = 1
# f_df["status"] = f_df["status"].astype(int)

f_df["category_id"] = f_df["category_id"].astype(int)
# f_df = f_df.rename(columns={"channel": "channel_title"})

# records = f_df[["channel_id", "channel_title",  "category_id", "status"]].to_dict(orient="records")
records = f_df[["channel_id",  "category_id"]].to_dict(orient="records")
print(records[:2])

for i in range(0, len(records), 100):
    data = records[i:i+100]
    await ChannelDAO.update_bulk(data)
    # await ChannelDAO.add_update_bulk(data, do_nothing=False)
    print(f"{i} / {len(records)}")

[{'channel_id': 'UCb1WBEEfA4e6FOLOKAH5j8w', 'category_id': 6}, {'channel_id': 'UCBZNnwQOBirwpeWncqTj_KQ', 'category_id': 6}]
0 / 394
100 / 394
200 / 394
300 / 394


In [53]:
from app.channel import ChannelStatDAO

# df_stat = pd.read_csv("whatstat_channels_stat.csv", sep='\t')
# df_stat.rename(columns={"period": "report_period", "subscribers": "subscriber_count", "views": "channel_view_count"}, inplace=True)
# df_stat["report_period"] = pd.to_datetime(df_stat["report_period"])
# df_stat["data_at"] = df_stat["report_period"]
# print(df_stat.columns)
records = df_stat.to_dict(orient="records")
print(len(records))
# print(records[0])
for i in range(0, len(records), 500):
    data = records[i:i+500]
    await ChannelStatDAO.add_bulk(data)
    print(f"{i} / {len(records)}")


22469
0 / 22469
500 / 22469
1000 / 22469
1500 / 22469
2000 / 22469
2500 / 22469
3000 / 22469
3500 / 22469
4000 / 22469
4500 / 22469
5000 / 22469
5500 / 22469
6000 / 22469
6500 / 22469
7000 / 22469
7500 / 22469
8000 / 22469
8500 / 22469
9000 / 22469
9500 / 22469
10000 / 22469
10500 / 22469
11000 / 22469
11500 / 22469
12000 / 22469
12500 / 22469
13000 / 22469
13500 / 22469
14000 / 22469
14500 / 22469
15000 / 22469
15500 / 22469
16000 / 22469
16500 / 22469
17000 / 22469
17500 / 22469
18000 / 22469
18500 / 22469
19000 / 22469
19500 / 22469
20000 / 22469
20500 / 22469
21000 / 22469
21500 / 22469
22000 / 22469


In [ ]:
# import datetime as dt
# dt.strptime("1 янв 2025 Ср", "%d %b %Y %a")
# df_stat["channel_view_count"] = df_stat["channel_view_count"].fillna(0)
df_stat[df_stat["channel_view_count"].isna()]

C:\Users\eremi\AppData\Local\Temp\ipykernel_14552\644973057.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_stat["channel_view_count"].fillna(0, inplace=True)


,channel_id,report_period,subscriber_count,channel_view_count,data_at
